In [1]:
# Cell 1: Imports
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, random_split

from gan_utils.io import H5ImageDataset
from gan.model import Generator, Discriminator, Encoder, DiscriminatorFeatures
from gan.trainer import Trainer

import matplotlib.pyplot as plt




In [2]:
# SECTION 2 — Utilities

def set_seed(seed: int = 42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def ensure_dir(path: str):
    if path and not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def make_dataloaders(df, h5_path, batch_size=32, workers=16, val_split=0.1):
    """
    df: pandas DataFrame with file_name column that matches keys in HDF5
    h5_path: path to HDF5 file containing images
    returns: train_loader, val_loader
    """
    dataset = H5ImageDataset(df, h5_path, transforms=None)
    n_total = len(dataset)
    n_val = int(val_split * n_total)
    n_train = n_total - n_val

    g = torch.Generator().manual_seed(123)
    train_ds, val_ds = random_split(dataset, [n_train, n_val], generator=g)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=workers,
        pin_memory=True,
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers,
        pin_memory=True,
    )
    return train_loader, val_loader


def build_models(n_z=256, image_size=512, in_channels=3):
    """
    Construct Generator and Discriminator.
    """
    gen = Generator(
        n_z=n_z,
        input_filt=512,
        final_size=image_size,
        out_channels=in_channels,
        norm=False,
        pool=False
    )

    disc = Discriminator(
        in_channels=in_channels,
        n_layers=6,
        input_size=image_size,
        norm=False,
        pool=False
    )

    return gen, disc


def train_gan(
    gen,
    disc,
    train_loader,
    val_loader,
    outdir,
    epochs=50,
    d_lr=1e-4,
    g_lr=1e-4,
    save_freq=10,
    lr_decay=None,
    decay_freq=5,
    device='cuda'
):
    """Train WGAN-GP using Trainer and save checkpoints + loss metrics."""
    ensure_dir(outdir)

    gen = gen.to(device)
    disc = disc.to(device)

    trainer = Trainer(
        generator=gen,
        discriminator=disc,
        savefolder=outdir,
        device=device
    )

    G_loss_ep, D_loss_ep = trainer.train(
        train_data=train_loader,
        val_data=val_loader,
        epochs=epochs,
        dsc_learning_rate=d_lr,
        gen_learning_rate=g_lr,
        save_freq=save_freq,
        lr_decay=lr_decay,
        decay_freq=decay_freq
    )

    return G_loss_ep, D_loss_ep


In [3]:
# SECTION 3 — Config

h5_path = "anomaly.h5"
parquet_path = "mask_labels_anomaly.gzip"
outdir = "./outputs"
gan_ckpt_dir = os.path.join(outdir, "gan_checkpoints")

batch_size = 32
workers = 16
val_split = 0.1
image_size = 512
n_z = 256
gan_epochs = 50
g_lr = 1e-4
d_lr = 1e-4

device = "cuda" if torch.cuda.is_available() else "cpu"
seed = 42

set_seed(seed)
ensure_dir(outdir)
ensure_dir(gan_ckpt_dir)


In [4]:
# SECTION 4 — Load metadata + dataloaders

df = pd.read_parquet(parquet_path)

train_loader, val_loader = make_dataloaders(
    df,
    h5_path,
    batch_size=batch_size,
    workers=workers,
    val_split=val_split,
)


In [5]:
# SECTION 5 — Build models

# === Rebuild models ===
gen, disc = build_models(n_z=n_z, image_size=image_size, in_channels=3)
gen = gen.to(device)
disc = disc.to(device)

# Quick sanity check
imgs = next(iter(train_loader))[0]
print("Real batch shape:", imgs.shape)

z = torch.randn(4, gen.n_z).to(device)
fake = gen(z)
print("Fake batch shape:", fake.shape)





Real batch shape: torch.Size([3, 512, 512])
Fake batch shape: torch.Size([4, 3, 512, 512])


In [6]:
# === Find latest checkpoints ===
import glob, os, torch

ckpts_g = sorted(glob.glob(os.path.join(gan_ckpt_dir, "generator_ep_*.pth")))
ckpts_d = sorted(glob.glob(os.path.join(gan_ckpt_dir, "discriminator_ep_*.pth")))

latest_g = ckpts_g[-1]
latest_d = ckpts_d[-1]

# Extract epoch number from filename
last_epoch = int(latest_g.split("_ep_")[1].split(".")[0])
print("Last completed epoch:", last_epoch)

# === Load weights ===
gen.load_state_dict(torch.load(latest_g, map_location=device))
disc.load_state_dict(torch.load(latest_d, map_location=device))

print("Loaded generator + discriminator.")


Last completed epoch: 50


/tmp/ipykernel_3986563/1529539410.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gen.load_state_dict(torch.load(latest_g, map_location=device))


Loaded generator + discriminator.


/tmp/ipykernel_3986563/1529539410.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  disc.load_state_dict(torch.load(latest_d, map_location=device))


In [7]:
# === Create Trainer ===
trainer = Trainer(
    generator=gen,
    discriminator=disc,
    savefolder=gan_ckpt_dir,
    device=device
)

# Tell Trainer where to resume
trainer.start = last_epoch + 1

print(f"Resuming from epoch {trainer.start}")

# === Resume training ===
G_loss_ep, D_loss_ep = trainer.train(
    train_data=train_loader,
    val_data=val_loader,
    epochs=last_epoch + 50,   # train 50 more epochs
    dsc_learning_rate=d_lr,
    gen_learning_rate=g_lr,
    save_freq=10
)

print("Finished resumed training.")


/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:40: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler_g = GradScaler()
/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:41: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler_d = GradScaler()


Resuming from epoch 51
Epoch 51 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Training:   0%|          | 0/2813 [00:00<?, ?it/s]/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:62: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:76: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validation: 100%|██████████| 313/313 [00:33<00:00,  9.38it/s, gen: 1.84e+03 disc: 4.72e+00 w_dist: 4.65e+00 grad_penalty: 6.73e-02]


Epoch 52 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Validation: 100%|██████████| 313/313 [00:40<00:00,  7.71it/s, gen: 1.15e+03 disc: -1.06e+00 w_dist: -1.08e+00 grad_penalty: 2.31e-02]


...

Epoch 99 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Validation: 100%|██████████| 313/313 [00:30<00:00, 10.37it/s, gen: -1.40e+03 disc: -1.50e+01 w_dist: -1.51e+01 grad_penalty: 4.84e-02]


Epoch 100 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Validation: 100%|██████████| 313/313 [00:30<00:00, 10.17it/s, gen: -3.22e+03 disc: -2.90e+01 w_dist: -3.19e+01 grad_penalty: 2.87e+00]


Saving to ./outputs/gan_checkpoints//generator_ep_100.pth and ./outputs/gan_checkpoints//discriminator_ep_100.pth
Finished resumed training.


In [8]:
# SECTION 7 — Plot GAN losses

plt.figure()
plt.plot(G_loss_ep, label="Gen loss", color="red")
plt.plot(D_loss_ep, label="Disc loss", color="blue")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outdir, "gan_losses.png"))
plt.close()


In [9]:
# Cell 8: Save final generator and discriminator

torch.save(gen.state_dict(), os.path.join(gan_ckpt_dir, "generator_final.pth"))
torch.save(disc.state_dict(), os.path.join(gan_ckpt_dir, "discriminator_final.pth"))

print("Done training GAN. Checkpoints in:", gan_ckpt_dir)


Done training GAN. Checkpoints in: ./outputs/gan_checkpoints


# 